In [2]:
from __future__ import  division, print_function, absolute_import
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mne
from sklearn.model_selection import cross_val_score
from mne.io import RawArray
from mne.epochs import concatenate_epochs
from mne import create_info, find_events, Epochs
from mne.channels import read_custom_montage
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from glob import glob
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.signal import welch
from mne import pick_types
import os.path
import scipy.io as sio
import time
from cnn_class import cnn
import warnings
from sklearn.metrics import classification_report, roc_auc_score, auc, roc_curve, f1_score
from RnnAttention.attention import attention
from scipy import interp
warnings.filterwarnings('ignore')
##########################
# create MNE readable file 
##########################
def creat_mne_raw_object(fname):
    """Create a mne raw instance from csv file"""
    # Read EEG file
    data = pd.read_csv(fname)
    
    # get chanel names
    ch_names = list(data.columns[1:])
    
    # read EEG standard montage from mne
    montage = 'standard_1005'

    # events file
    ev_fname = fname.replace('_data','_events')
    # read event file
    events = pd.read_csv(ev_fname)
    events_names = events.columns[1:]
    events_data = np.array(events[events_names]).T
    
    # concatenate event file and data
    data = np.concatenate((1e-6*np.array(data[ch_names]).T,events_data))        
    
    # define channel type, the first is EEG, the last 6 are stimulations
    ch_type = ['eeg']*len(ch_names) + ['stim']*6
    
    # create and populate MNE info structure
    ch_names.extend(events_names)
    info = create_info(ch_names,sfreq=500.0,ch_types=ch_type,montage = montage)
    # info.set_montage(montage)
    #info['filename'] = fname
    
    # create raw object 
    raw = RawArray(data,info,verbose=False)
    return raw


In [3]:
####################################################################
# Data preprocessing, band_pass, event related 1.5s + 1.5s featuring
####################################################################
def data_gen(subjects):    
    X_train=[]
    y_train=[]
    n = 0

    for i in range(len(subjects)):
        n+=1
        subject = subjects[i]
        epochs_Replace_tot = []
        epochs_Replace_rest_tot = []
        epochs_HandStart_tot = []
        epochs_HandStart_rest_tot = []

        fnames =  glob('../train/subj%d_series*_data.csv' % (subject))

        y = []
        for i,fname in enumerate(fnames):

            # read data 
            raw = creat_mne_raw_object(fname)
            #raw.plot(block=True)
            # pick eeg signal
            picks = pick_types(raw.info,eeg=True)

            # Filter data for alpha frequency and beta band
            # Note that MNE implement a zero phase (filtfilt) filtering not compatible
            # with the rule of future data.
            raw.filter(7,35, picks=picks, method='iir', n_jobs=-1, verbose=False)

            #id = 'Replace'
            # get event posision corresponding to Replace
            events = find_events(raw,stim_channel='Replace', verbose=False)
            # epochs signal for 1.5 second before the movement
            epochs = Epochs(raw, events, {'during' : 1}, -2, -0.5, proj=False, picks=picks, baseline=None, preload=True, verbose=False)

            epochs_Replace_tot.append(epochs)


            # epochs signal for 1.5 second after the movement, this correspond to the 
            # rest period.
            epochs_rest = Epochs(raw, events, {'after' : 1}, 0.5, 2, proj=False, picks=picks, baseline=None, preload=True, verbose=False)

            # Workaround to be able to concatenate epochs
            epochs_rest.shift_time(-2.5, relative = True)

            epochs_Replace_rest_tot.append(epochs_rest)

            # id = 'HandStart'
            # get event posision corresponding to HandStart
            events = find_events(raw,stim_channel='HandStart', verbose=False)
            # epochs signal for 1.5 second before the movement
            epochs = Epochs(raw, events, {'during' : 1}, 0.5, 2, proj=False, picks=picks, baseline=None, preload=True, verbose=False)

            epochs_HandStart_tot.append(epochs)

            # epochs signal for 1.5 second after the movement, this correspond to the 
            # rest period.
            epochs_rest = Epochs(raw, events, {'before' : 1}, -2, -0.5, proj=False, picks=picks, baseline=None, preload=True, verbose=False)

            # Workaround to be able to concatenate epochs
            epochs_rest.shift_time(2.5, relative = True)

            epochs_HandStart_rest_tot.append(epochs_rest)


        epochs_Replace = concatenate_epochs(epochs_Replace_tot)
        epochs_Replace_rest = concatenate_epochs(epochs_Replace_rest_tot)
        epochs_HandStart = concatenate_epochs(epochs_HandStart_tot)
        epochs_HandStart_rest = concatenate_epochs(epochs_HandStart_rest_tot)
    
        # epochs.plot_psd( fmin=2.,fmax=40.,average=True, spatial_colors=True)
        # epochs_rest.plot_psd( fmin=2.,fmax=40.,average=True, spatial_colors=True)

        # epochs.plot_psd_topomap(ch_type='eeg',normalize=True)
        # epochs_rest.plot_psd_topomap(ch_type='eeg',normalize=True)

        #get data 
        X_Replace = epochs_Replace.get_data()
        X_Replace_rest = epochs_Replace_rest.get_data()
        X_Replace_var = np.concatenate((X_Replace,X_Replace_rest),axis=1)

        X_HandStart = epochs_HandStart.get_data()
        X_HandStart_rest = epochs_HandStart_rest.get_data()
        X_HandStart_var = np.concatenate((X_HandStart_rest,X_HandStart),axis=1)

        X = np.concatenate((X_Replace,X_HandStart),axis=0)
        #y = np.array(y)
        print ("subject",subject,X.shape)

        if n == 1:
            X_train = X

        else:
            X_train=np.append(X_train,X,axis =0)

        y = np.concatenate((np.ones(260),np.zeros(260)),axis=0)
        y_train = np.concatenate((y_train,y),axis=0)
        print ("data stack shape:",X_train.shape,y_train.shape)

    # generate feature dataset for next process, it contains (HandStar,rest),(Replace,rest) 
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    print ('return data shape: ',X_train.shape,y_train.shape)
    return (X_train, y_train)


In [4]:
for j in range(8,13):
    train_subject = [k for k in range(1,13) if k != j]
    test_subject = [j]
    train_X,train_y = data_gen(train_subject)
    test_X,test_y = data_gen(test_subject)
    
    idx = list(range(len(train_y)))
    np.random.shuffle(idx)
    train_X = train_X[idx]
    train_y = train_y[idx]
    
    idx = list(range(len(test_y)))
    np.random.shuffle(idx)
    test_X = test_X[idx]
    test_y = test_y[idx]    
    
    #sio.savemat('../TACR_noRest_independent/T-set_sub'+str(j)+'.mat', {"train_x": train_X, "train_y": train_y, "test_x": test_X, "test_y": test_y})
    # train_X preprocess
    X_inputs = np.transpose(train_X, [1, 0, 2])
    X_inputs=X_inputs.reshape((X_inputs.shape[0],(X_inputs.shape[1]*X_inputs.shape[2])))
    X_inputs = np.transpose(X_inputs, [1, 0])
    X_inputs = X_inputs*100000
    print ('X_inputs',X_inputs.shape)
    # test_X preprocess
    X_inputs1 = np.transpose(test_X, [1, 0, 2])
    X_inputs1 = X_inputs1.reshape((X_inputs1.shape[0],(X_inputs1.shape[1]*X_inputs1.shape[2])))
    X_inputs1 = np.transpose(X_inputs1, [1, 0])
    X_inputs1 = X_inputs1*100000
    print ('X_inputs1',X_inputs1.shape)

    # X_inputs and Y_targets are np.array, while X,Y are tf.tensor class

    learning_rate = 0.01
    training_epochs = 5
    batch_size = 751
    display_step = 1
    n_input = 32
    X = tf.placeholder("float", [None, n_input])

    n_hidden_1 = 88
    n_hidden_2 = 44
    n_hidden_3 = 22

    weights = {
        'encoder_h1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1], )),
        'encoder_h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2], )),
        'encoder_h3': tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3], )),
        'decoder_h1': tf.Variable(tf.truncated_normal([n_hidden_3, n_hidden_2], )),
        'decoder_h2': tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_1], )),
        'decoder_h3': tf.Variable(tf.truncated_normal([n_hidden_1, n_input], )),
    }
    biases = {
        'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'encoder_b3': tf.Variable(tf.random_normal([n_hidden_3])),
        'decoder_b1': tf.Variable(tf.random_normal([n_hidden_2])),
        'decoder_b2': tf.Variable(tf.random_normal([n_hidden_1])),
        'decoder_b3': tf.Variable(tf.random_normal([n_input])),
    }


    def encoder(x):
        layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                       biases['encoder_b1']))
        layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                       biases['encoder_b2']))
        layer_3 = tf.add(tf.matmul(layer_2, weights['encoder_h3']),
                         biases['encoder_b3'])

        return layer_3


    def decoder(x):
        layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                       biases['decoder_b1']))
        layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                       biases['decoder_b2']))
        layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['decoder_h3']),
                                       biases['decoder_b3']))
        return layer_3

    # iteration
    encoder_op = encoder(X)
    decoder_op = decoder(encoder_op)

    y_pred = decoder_op
    y_true = X

    cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2)) # cost func. MSE
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    encoder_result = []
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        total_batch = int(X_inputs.shape[0] / batch_size)
        for epoch in range(training_epochs):
            for i in range(total_batch):
                offset = (i * batch_size) 
                batch_xs = X_inputs[offset:(offset + batch_size), :] 
                #batch_ys = Y_targets[offset:(offset + batch_size), :]
                print ('.',end = '')
                _, c = sess.run([optimizer, cost], feed_dict={X: batch_xs})
            if epoch % display_step == 0:
                print()
                print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(c))
        print("Optimization Finished!")

        # build encoder_result array as feature vector
        train_X=sess.run(encoder_op, feed_dict={X: X_inputs})      
        train_X=train_X.reshape(int(train_X.shape[0]/751),751,22)
        train_X=np.transpose(train_X,[0,2,1])

        test_X=sess.run(encoder_op, feed_dict={X: X_inputs1})      
        test_X=test_X.reshape(int(test_X.shape[0]/751),751,22)
        test_X=np.transpose(test_X,[0,2,1])

        print (train_X.shape, test_X.shape, train_y.shape,test_y.shape)
        display (test_X)

    sess.close()

    test_y	= test_y.ravel()
    train_y = train_y.ravel()
    display (test_y)

    train_y = np.asarray(pd.get_dummies(train_y), dtype = np.int8)
    test_y = np.asarray(pd.get_dummies(test_y), dtype = np.int8)

    #############
    # Set window
    #############

    window_size = 100
    step = 50
    n_channel = 22

    def windows(data, size, step):
        start = 0
        while ((start+size) < data.shape[0]):
            yield int(start), int(start + size)
            start += step

    def segment_signal_without_transition(data, window_size, step):
        segments = []
        for (start, end) in windows(data, window_size, step):
            if(len(data[start:end]) == window_size):
                segments = segments + [data[start:end]]
        return np.array(segments)

    def segment_dataset(X, window_size, step):
        win_x = []
        for i in range(X.shape[0]):
            win_x = win_x + [segment_signal_without_transition(X[i], window_size, step)]
        win_x = np.array(win_x)
        return win_x

    train_raw_x = np.transpose(train_X, [0, 2, 1])
    test_raw_x = np.transpose(test_X, [0, 2, 1])


    train_win_x = segment_dataset(train_raw_x, window_size, step)
    print("train_win_x shape: ", train_win_x.shape)
    test_win_x = segment_dataset(test_raw_x, window_size, step)
    print("test_win_x shape: ", test_win_x.shape)

    # [trial, window, channel, time_length]
    train_win_x = np.transpose(train_win_x, [0, 1, 3, 2])
    print("train_win_x shape: ", train_win_x.shape)

    test_win_x = np.transpose(test_win_x, [0, 1, 3, 2])
    print("test_win_x shape: ", test_win_x.shape)


    # [trial, window, channel, time_length, 1]
    train_x = np.expand_dims(train_win_x, axis = 4)
    test_x = np.expand_dims(test_win_x, axis = 4)

    num_timestep = train_x.shape[1]
    ###########################################################################
    # set model parameters
    ###########################################################################
    # kernel parameter
    kernel_height_1st	= 22
    kernel_width_1st 	= 45

    kernel_stride		= 1

    conv_channel_num	= 40

    # pooling parameter
    pooling_height_1st 	= 1
    pooling_width_1st 	= 56

    pooling_stride_1st = 10

    # full connected parameter
    attention_size = 512
    n_hidden_state = 64

    ###########################################################################
    # set dataset parameters
    ###########################################################################
    # input channel
    input_channel_num = 1

    # input height 
    input_height = train_x.shape[2]

    # input width
    input_width = train_x.shape[3]

    # prediction class
    num_labels = 2
    ###########################################################################
    # set training parameters
    ###########################################################################
    # set learning rate
    learning_rate = 1e-4

    # set maximum traing epochs
    training_epochs = 100

    # set batch size
    batch_size = 10

    # set dropout probability
    dropout_prob = 0.5

    # set train batch number per epoch
    batch_num_per_epoch = train_x.shape[0]//batch_size

    # instance cnn class
    padding = 'VALID'

    cnn_2d = cnn(padding=padding)

    # input placeholder
    X = tf.placeholder(tf.float32, shape=[None, input_height, input_width, input_channel_num], name = 'X')
    Y = tf.placeholder(tf.float32, shape=[None, num_labels], name = 'Y')
    train_phase = tf.placeholder(tf.bool, name = 'train_phase')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    # first CNN layer
    conv_1 = cnn_2d.apply_conv2d(X, kernel_height_1st, kernel_width_1st, input_channel_num, conv_channel_num, kernel_stride, train_phase)
    print("conv 1 shape: ", conv_1.get_shape().as_list())
    pool_1 = cnn_2d.apply_max_pooling(conv_1, pooling_height_1st, pooling_width_1st, pooling_stride_1st)
    print("pool 1 shape: ", pool_1.get_shape().as_list())

    pool1_shape = pool_1.get_shape().as_list()
    pool1_flat = tf.reshape(pool_1, [-1, pool1_shape[1]*pool1_shape[2]*pool1_shape[3]])

    fc_drop = tf.nn.dropout(pool1_flat, keep_prob)	

    lstm_in = tf.reshape(fc_drop, [-1, num_timestep, pool1_shape[1]*pool1_shape[2]*pool1_shape[3]])

    ################
    # pipline design
    #################

    ########################## RNN ########################
    cells = []
    for _ in range(2):
        cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_state, forget_bias=1.0, state_is_tuple=True)
        cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
        cells.append(cell)
    lstm_cell = tf.contrib.rnn.MultiRNNCell(cells)

    init_state = lstm_cell.zero_state(batch_size, dtype=tf.float32)

    # output ==> [batch, step, n_hidden_state]
    rnn_op, states = tf.nn.dynamic_rnn(lstm_cell, lstm_in, initial_state=init_state, time_major=False)

    ########################## attention ########################
    with tf.name_scope('Attention_layer'):
        attention_op, alphas = attention(rnn_op, attention_size, time_major = False, return_alphas=True)

    attention_drop = tf.nn.dropout(attention_op, keep_prob)	

    ########################## readout ########################
    y_ = cnn_2d.apply_readout(attention_drop, rnn_op.shape[2].value, num_labels)

    # probability prediction 
    y_prob = tf.nn.softmax(y_, name = "y_prob")

    # class prediction 
    y_pred = tf.argmax(y_prob, 1, name = "y_pred")

    ########################## loss and optimizer ########################
    # cross entropy cost function
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=Y), name = 'loss')


    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
    # set training SGD optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

    # get correctly predicted object
    correct_prediction = tf.equal(tf.argmax(tf.nn.softmax(y_), 1), tf.argmax(Y, 1))

    ########################## define accuracy ########################
    # calculate prediction accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name = 'accuracy')

    #############
    # train test 
    #############
    def multiclass_roc_auc_score(y_true, y_score):
        assert y_true.shape == y_score.shape
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        n_classes = y_true.shape[1]
        # compute ROC curve and ROC area for each class
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
        # compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(y_true.ravel(), y_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        # compute macro-average ROC curve and ROC area
        # First aggregate all false probtive rates
        all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
        # Then interpolate all ROC curves at this points
        mean_tpr = np.zeros_like(all_fpr)
        for i in range(n_classes):
            mean_tpr += interp(all_fpr, fpr[i], tpr[i])
        # Finally average it and compute AUC
        mean_tpr /= n_classes
        fpr["macro"] = all_fpr
        tpr["macro"] = mean_tpr
        roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
        return roc_auc
    # run with gpu memory growth
    config = tf.ConfigProto()
    # config.gpu_options.allow_growth = True

    train_acc = []
    test_acc = []
    best_test_acc = []
    train_loss = []
    with tf.Session(config=config) as session:
        session.run(tf.global_variables_initializer())
        best_acc = 0
        for epoch in range(training_epochs):
            pred_test = np.array([])
            true_test = []
            prob_test = []
            ########################## training process ########################
            for b in range(batch_num_per_epoch):
                offset = (b * batch_size) % (train_y.shape[0] - batch_size) 
                batch_x = train_x[offset:(offset + batch_size), :, :, :, :]
                batch_x = batch_x.reshape([len(batch_x)*num_timestep, n_channel, window_size, 1])
                batch_y = train_y[offset:(offset + batch_size), :]
                _, c = session.run([optimizer, cost], feed_dict={X: batch_x, Y: batch_y, keep_prob: 1-dropout_prob, train_phase: True})
            # calculate train and test accuracy after each training epoch
            if(epoch%1 == 0):
                train_accuracy 	= np.zeros(shape=[0], dtype=float)
                test_accuracy	= np.zeros(shape=[0], dtype=float)
                train_l 		= np.zeros(shape=[0], dtype=float)
                test_l			= np.zeros(shape=[0], dtype=float)
                # calculate train accuracy after each training epoch
                for i in range(batch_num_per_epoch):
                    ########################## prepare training data ########################
                    offset = (i * batch_size) % (train_y.shape[0] - batch_size) 
                    train_batch_x = train_x[offset:(offset + batch_size), :, :, :]
                    train_batch_x = train_batch_x.reshape([len(train_batch_x)*num_timestep, n_channel, window_size, 1])
                    train_batch_y = train_y[offset:(offset + batch_size), :]

                    ########################## calculate training results ########################
                    train_a, train_c = session.run([accuracy, cost], feed_dict={X: train_batch_x, Y: train_batch_y, keep_prob: 1.0, train_phase: False})

                    train_l = np.append(train_l, train_c)
                    train_accuracy = np.append(train_accuracy, train_a)
                print("("+time.asctime(time.localtime(time.time()))+") Epoch: ", epoch+1, " Training Cost: ", np.mean(train_l), "Training Accuracy: ", np.mean(train_accuracy))
                train_acc = train_acc + [np.mean(train_accuracy)]
                train_loss = train_loss + [np.mean(train_l)]
                # calculate test accuracy after each training epoch
                for j in range(batch_num_per_epoch):
                    ########################## prepare test data ########################
                    offset = (j * batch_size) % (test_y.shape[0] - batch_size) 
                    test_batch_x = test_x[offset:(offset + batch_size), :, :, :]
                    test_batch_x = test_batch_x.reshape([len(test_batch_x)*num_timestep, n_channel, window_size, 1])
                    test_batch_y = test_y[offset:(offset + batch_size), :]

                    ########################## calculate test results ########################
                    test_a, test_c, prob_v, pred_v = session.run([accuracy, cost, y_prob, y_pred], feed_dict={X: test_batch_x, Y: test_batch_y, keep_prob: 1.0, train_phase: False})

                    test_accuracy = np.append(test_accuracy, test_a)
                    test_l = np.append(test_l, test_c)
                    pred_test = np.append(pred_test, pred_v)
                    true_test.append(test_batch_y)
                    prob_test.append(prob_v)
                if np.mean(test_accuracy) > best_acc :
                    best_acc = np.mean(test_accuracy)
                true_test = np.array(true_test).reshape([-1, num_labels])
                prob_test = np.array(prob_test).reshape([-1, num_labels])
                auc_roc_test = multiclass_roc_auc_score(y_true=true_test, y_score=prob_test)
                f1 = f1_score (y_true=np.argmax(true_test, axis = 1), y_pred=pred_test, average = 'macro')
                print("("+time.asctime(time.localtime(time.time()))+") Epoch: ", epoch+1, "Test Cost: ", np.mean(test_l), 
                                                                                          "Test Accuracy: ", np.mean(test_accuracy), 
                                                                                          "Test f1: ", f1, 
                                                                                          "Test AUC: ", auc_roc_test['macro'], "\n")



DigMontage is a superset of info. 311 in DigMontage will be ignored. The ignored channels are: {'PPO8', 'P9', 'CP4h', 'T4', 'FFC4h', 'FFT8h', 'AF1h', 'POO10', 'PPO10', 'FT9', 'AF3h', 'FCCz', 'CP6h', 'AFF4', 'C4h', 'F3h', 'AFF8h', 'CPP5h', 'PO1h', 'I2h', 'AFF10', 'F2', 'O2h', 'TTP10', 'T9', 'PPO9', 'F5', 'I2', 'FFC5', 'TPP10', 'FFT10', 'T10h', 'PO7h', 'TP9h', 'P5h', 'CP3', 'CPP1', 'POO10h', 'AFz', 'FC2h', 'PPO8h', 'F6', 'Fpz', 'CCP6', 'C6h', 'AFp9', 'AFp5', 'AFFz', 'TPP8', 'F8h', 'AF9', 'AFp2h', 'AFF5h', 'AF8', 'POO5h', 'POO1h', 'AFF6h', 'P9h', 'PPO2h', 'PPO5', 'F1', 'POO2', 'TTP8', 'AF7', 'T8h', 'PO6', 'CPP1h', 'CPP4', 'AFp2', 'PPO3h', 'F6h', 'CPP4h', 'POOz', 'F5h', 'AFp3', 'AFp7h', 'AFp10', 'POO7h', 'CCP1', 'POO6', 'AFp8', 'CPP3h', 'AFF5', 'CPP2h', 'FFT8', 'CCP6h', 'TP10h', 'OIz', 'AFp3h', 'CCP4h', 'PPO1', 'FCC6', 'PO4', 'PO8', 'TTP9h', 'FFC5h', 'AF3', 'P1h', 'CCP5h', 'AFF7', 'FFC1', 'FT7h', 'P2h', 'P8h', 'AFp10h', 'AFp5h', 'CPPz', 'AFF1', 'PO10h', 'A2', 'AF4', 'PO9h', 'FTT9h', 'AF1',

<ipython-input-2-de6cec34629d>:52: RuntimeWarning: 1 channel names are too long, have been truncated to 15 characters:
['BothStartLoadPhase']
  info = create_info(ch_names,sfreq=500.0,ch_types=ch_type,montage = montage)


DigMontage is a superset of info. 311 in DigMontage will be ignored. The ignored channels are: {'PPO8', 'P9', 'CP4h', 'T4', 'FFC4h', 'FFT8h', 'AF1h', 'POO10', 'PPO10', 'FT9', 'AF3h', 'FCCz', 'CP6h', 'AFF4', 'C4h', 'F3h', 'AFF8h', 'CPP5h', 'PO1h', 'I2h', 'AFF10', 'F2', 'O2h', 'TTP10', 'T9', 'PPO9', 'F5', 'I2', 'FFC5', 'TPP10', 'FFT10', 'T10h', 'PO7h', 'TP9h', 'P5h', 'CP3', 'CPP1', 'POO10h', 'AFz', 'FC2h', 'PPO8h', 'F6', 'Fpz', 'CCP6', 'C6h', 'AFp9', 'AFp5', 'AFFz', 'TPP8', 'F8h', 'AF9', 'AFp2h', 'AFF5h', 'AF8', 'POO5h', 'POO1h', 'AFF6h', 'P9h', 'PPO2h', 'PPO5', 'F1', 'POO2', 'TTP8', 'AF7', 'T8h', 'PO6', 'CPP1h', 'CPP4', 'AFp2', 'PPO3h', 'F6h', 'CPP4h', 'POOz', 'F5h', 'AFp3', 'AFp7h', 'AFp10', 'POO7h', 'CCP1', 'POO6', 'AFp8', 'CPP3h', 'AFF5', 'CPP2h', 'FFT8', 'CCP6h', 'TP10h', 'OIz', 'AFp3h', 'CCP4h', 'PPO1', 'FCC6', 'PO4', 'PO8', 'TTP9h', 'FFC5h', 'AF3', 'P1h', 'CCP5h', 'AFF7', 'FFC1', 'FT7h', 'P2h', 'P8h', 'AFp10h', 'AFp5h', 'CPPz', 'AFF1', 'PO10h', 'A2', 'AF4', 'PO9h', 'FTT9h', 'AF1',

<ipython-input-2-de6cec34629d>:52: RuntimeWarning: 1 channel names are too long, have been truncated to 15 characters:
['BothStartLoadPhase']
  info = create_info(ch_names,sfreq=500.0,ch_types=ch_type,montage = montage)


DigMontage is a superset of info. 311 in DigMontage will be ignored. The ignored channels are: {'PPO8', 'P9', 'CP4h', 'T4', 'FFC4h', 'FFT8h', 'AF1h', 'POO10', 'PPO10', 'FT9', 'AF3h', 'FCCz', 'CP6h', 'AFF4', 'C4h', 'F3h', 'AFF8h', 'CPP5h', 'PO1h', 'I2h', 'AFF10', 'F2', 'O2h', 'TTP10', 'T9', 'PPO9', 'F5', 'I2', 'FFC5', 'TPP10', 'FFT10', 'T10h', 'PO7h', 'TP9h', 'P5h', 'CP3', 'CPP1', 'POO10h', 'AFz', 'FC2h', 'PPO8h', 'F6', 'Fpz', 'CCP6', 'C6h', 'AFp9', 'AFp5', 'AFFz', 'TPP8', 'F8h', 'AF9', 'AFp2h', 'AFF5h', 'AF8', 'POO5h', 'POO1h', 'AFF6h', 'P9h', 'PPO2h', 'PPO5', 'F1', 'POO2', 'TTP8', 'AF7', 'T8h', 'PO6', 'CPP1h', 'CPP4', 'AFp2', 'PPO3h', 'F6h', 'CPP4h', 'POOz', 'F5h', 'AFp3', 'AFp7h', 'AFp10', 'POO7h', 'CCP1', 'POO6', 'AFp8', 'CPP3h', 'AFF5', 'CPP2h', 'FFT8', 'CCP6h', 'TP10h', 'OIz', 'AFp3h', 'CCP4h', 'PPO1', 'FCC6', 'PO4', 'PO8', 'TTP9h', 'FFC5h', 'AF3', 'P1h', 'CCP5h', 'AFF7', 'FFC1', 'FT7h', 'P2h', 'P8h', 'AFp10h', 'AFp5h', 'CPPz', 'AFF1', 'PO10h', 'A2', 'AF4', 'PO9h', 'FTT9h', 'AF1',

...

DigMontage is a superset of info. 311 in DigMontage will be ignored. The ignored channels are: {'PPO8', 'P9', 'CP4h', 'T4', 'FFC4h', 'FFT8h', 'AF1h', 'POO10', 'PPO10', 'FT9', 'AF3h', 'FCCz', 'CP6h', 'AFF4', 'C4h', 'F3h', 'AFF8h', 'CPP5h', 'PO1h', 'I2h', 'AFF10', 'F2', 'O2h', 'TTP10', 'T9', 'PPO9', 'F5', 'I2', 'FFC5', 'TPP10', 'FFT10', 'T10h', 'PO7h', 'TP9h', 'P5h', 'CP3', 'CPP1', 'POO10h', 'AFz', 'FC2h', 'PPO8h', 'F6', 'Fpz', 'CCP6', 'C6h', 'AFp9', 'AFp5', 'AFFz', 'TPP8', 'F8h', 'AF9', 'AFp2h', 'AFF5h', 'AF8', 'POO5h', 'POO1h', 'AFF6h', 'P9h', 'PPO2h', 'PPO5', 'F1', 'POO2', 'TTP8', 'AF7', 'T8h', 'PO6', 'CPP1h', 'CPP4', 'AFp2', 'PPO3h', 'F6h', 'CPP4h', 'POOz', 'F5h', 'AFp3', 'AFp7h', 'AFp10', 'POO7h', 'CCP1', 'POO6', 'AFp8', 'CPP3h', 'AFF5', 'CPP2h', 'FFT8', 'CCP6h', 'TP10h', 'OIz', 'AFp3h', 'CCP4h', 'PPO1', 'FCC6', 'PO4', 'PO8', 'TTP9h', 'FFC5h', 'AF3', 'P1h', 'CCP5h', 'AFF7', 'FFC1', 'FT7h', 'P2h', 'P8h', 'AFp10h', 'AFp5h', 'CPPz', 'AFF1', 'PO10h', 'A2', 'AF4', 'PO9h', 'FTT9h', 'AF1',

<ipython-input-2-de6cec34629d>:52: RuntimeWarning: 1 channel names are too long, have been truncated to 15 characters:
['BothStartLoadPhase']
  info = create_info(ch_names,sfreq=500.0,ch_types=ch_type,montage = montage)


DigMontage is a superset of info. 311 in DigMontage will be ignored. The ignored channels are: {'PPO8', 'P9', 'CP4h', 'T4', 'FFC4h', 'FFT8h', 'AF1h', 'POO10', 'PPO10', 'FT9', 'AF3h', 'FCCz', 'CP6h', 'AFF4', 'C4h', 'F3h', 'AFF8h', 'CPP5h', 'PO1h', 'I2h', 'AFF10', 'F2', 'O2h', 'TTP10', 'T9', 'PPO9', 'F5', 'I2', 'FFC5', 'TPP10', 'FFT10', 'T10h', 'PO7h', 'TP9h', 'P5h', 'CP3', 'CPP1', 'POO10h', 'AFz', 'FC2h', 'PPO8h', 'F6', 'Fpz', 'CCP6', 'C6h', 'AFp9', 'AFp5', 'AFFz', 'TPP8', 'F8h', 'AF9', 'AFp2h', 'AFF5h', 'AF8', 'POO5h', 'POO1h', 'AFF6h', 'P9h', 'PPO2h', 'PPO5', 'F1', 'POO2', 'TTP8', 'AF7', 'T8h', 'PO6', 'CPP1h', 'CPP4', 'AFp2', 'PPO3h', 'F6h', 'CPP4h', 'POOz', 'F5h', 'AFp3', 'AFp7h', 'AFp10', 'POO7h', 'CCP1', 'POO6', 'AFp8', 'CPP3h', 'AFF5', 'CPP2h', 'FFT8', 'CCP6h', 'TP10h', 'OIz', 'AFp3h', 'CCP4h', 'PPO1', 'FCC6', 'PO4', 'PO8', 'TTP9h', 'FFC5h', 'AF3', 'P1h', 'CCP5h', 'AFF7', 'FFC1', 'FT7h', 'P2h', 'P8h', 'AFp10h', 'AFp5h', 'CPPz', 'AFF1', 'PO10h', 'A2', 'AF4', 'PO9h', 'FTT9h', 'AF1',

<ipython-input-2-de6cec34629d>:52: RuntimeWarning: 1 channel names are too long, have been truncated to 15 characters:
['BothStartLoadPhase']
  info = create_info(ch_names,sfreq=500.0,ch_types=ch_type,montage = montage)


subject 12 (520, 32, 751)
data stack shape: (520, 32, 751) (520,)
return data shape:  (520, 32, 751) (520,)
